In [25]:
!ls

CSV_data_preprocessing	Functions   Payload-Byte-logo.jpg  README.md
Data			License.md  Pipeline.ipynb	   requirements.txt


In [26]:
!unzip -q Payload-Byte-main.zip -d payload_byte


unzip:  cannot find or open Payload-Byte-main.zip, Payload-Byte-main.zip.zip or Payload-Byte-main.zip.ZIP.


In [27]:
!ls

CSV_data_preprocessing	Functions   Payload-Byte-logo.jpg  README.md
Data			License.md  Pipeline.ipynb	   requirements.txt


In [28]:
!ls payload_byte/Payload-Byte-main


ls: cannot access 'payload_byte/Payload-Byte-main': No such file or directory


In [29]:
!pip install scapy

In [30]:
import os

os.chdir("/content/payload_byte/Payload-Byte-main")
print("Current working directory:", os.getcwd())
print("Contents here:", os.listdir("."))


Current working directory: /content/payload_byte/Payload-Byte-main
Contents here: ['Data', 'Functions', 'CSV_data_preprocessing', 'Payload-Byte-logo.jpg', 'requirements.txt', 'README.md', '.gitignore', 'Pipeline.ipynb', 'License.md']


In [31]:
import os

print("Contents of Data/:")
print(os.listdir("Data"))

# If there is a UNSW subdirectory, list that too
unsw_path = "Data/UNSW-NB15"
if os.path.exists(unsw_path):
    print("\nContents of Data/UNSW-NB15/:")
    print(os.listdir(unsw_path))


Contents of Data/:
['Feature_Vector.png', 'readme.md']


In [32]:
import os
print(os.listdir("/content"))


['.config', 'UNSW-NB15_4.csv', 'payload_byte', 'UNSW-NB15_2.csv', 'Payload-Byte-main.zip', '1.pcap', 'UNSW-NB15_3.csv', 'UNSW-NB15_1.csv', 'sample_data']


In [34]:
import pandas as pd

# Load and merge the 4 UNSW CSVs
csv1 = pd.read_csv("/content/UNSW-NB15_1.csv")
csv2 = pd.read_csv("/content/UNSW-NB15_2.csv")
csv3 = pd.read_csv("/content/UNSW-NB15_3.csv")
csv4 = pd.read_csv("/content/UNSW-NB15_4.csv")

df_flow = pd.concat([csv1, csv2, csv3, csv4], ignore_index=True)

print("Total flow records:", len(df_flow))
df_flow.head()


/tmp/ipython-input-3207433459.py:4: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  csv1 = pd.read_csv("/content/UNSW-NB15_1.csv")
/tmp/ipython-input-3207433459.py:5: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  csv2 = pd.read_csv("/content/UNSW-NB15_2.csv")


Total flow records: 2540043


,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,...,456.043567,15.530109,1424250009,1424250009.1,3.943843,4.912488,0.00059,0.000473,0.000117,4
0,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528.0,304.0,31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146.0,178.0,31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132.0,164.0,31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146.0,178.0,31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,59.166.0.0,32119,149.171.126.9,111,udp,CON,0.078339,568.0,312.0,31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
from scapy.all import PcapReader, UDP, TCP, IP, ARP
import time

packets = []
max_packets = 20000   # you can increase later if needed

reader = PcapReader("/content/1.pcap")

count = 0
start = time.time()

for pkt in reader:
    if IP in pkt or ARP in pkt:
        pkt_info = {}

        # If ARP
        if ARP in pkt:
            pkt_info["srcip"] = pkt[ARP].psrc
            pkt_info["dstip"] = pkt[ARP].pdst
            pkt_info["sport"] = None
            pkt_info["dsport"] = None
            pkt_info["protocol"] = "arp"
            pkt_info["ttl"] = None
        # If IP
        elif IP in pkt:
            pkt_info["srcip"] = pkt[IP].src
            pkt_info["dstip"] = pkt[IP].dst
            pkt_info["ttl"] = pkt[IP].ttl

            # Protocol
            proto = pkt[IP].proto
            if proto == 6:
                pkt_info["protocol"] = "tcp"
                pkt_info["sport"] = pkt[TCP].sport if TCP in pkt else None
                pkt_info["dsport"] = pkt[TCP].dport if TCP in pkt else None
            elif proto == 17:
                pkt_info["protocol"] = "udp"
                pkt_info["sport"] = pkt[UDP].sport if UDP in pkt else None
                pkt_info["dsport"] = pkt[UDP].dport if UDP in pkt else None
            else:
                pkt_info["protocol"] = str(proto)
                pkt_info["sport"] = None
                pkt_info["dsport"] = None

        # Timestamp
        pkt_info["stime"] = float(pkt.time)

        # Add sequential packet number
        pkt_info["frame_num"] = count

        # Payload (hex)
        try:
            payload_bytes = bytes(pkt.payload.payload)
            pkt_info["payload"] = payload_bytes.hex() if payload_bytes else "0"
        except:
            pkt_info["payload"] = "0"

        packets.append(pkt_info)
        count += 1

    if count >= max_packets:
        break

end = time.time()

print("Parsed packets:", count)
print("Time taken:", end - start)


Parsed packets: 20000
Time taken: 7.780154228210449


In [36]:
# Turn parsed packets into a DataFrame
df_pcap = pd.DataFrame(packets)

print("Packet columns:", df_pcap.columns.tolist())
print("Number of packets:", len(df_pcap))

df_pcap.head()


Packet columns: ['srcip', 'dstip', 'ttl', 'protocol', 'sport', 'dsport', 'stime', 'frame_num', 'payload']
Number of packets: 20000


,srcip,dstip,ttl,protocol,sport,dsport,stime,frame_num,payload
0,10.40.85.1,224.0.0.5,1.0,89,NaN,NaN,1.421927e+09,0,0201002cc0a8f1f300000000ead8000000000000000000...
1,10.40.182.1,224.0.0.5,1.0,89,NaN,NaN,1.421927e+09,1,0201002cc0a8f1f30000000089d8000000000000000000...
2,192.168.241.243,192.168.241.243,64.0,1,NaN,NaN,1.421927e+09,2,030154460000000045000148000040004011d52dc0a8f1...
3,10.40.85.1,224.0.0.5,1.0,89,NaN,NaN,1.421927e+09,3,0201002cc0a8f1f300000000ead8000000000000000000...
4,10.40.182.1,224.0.0.5,1.0,89,NaN,NaN,1.421927e+09,4,0201002cc0a8f1f30000000089d8000000000000000000...


In [37]:
# --- Clean df_pcap for matching ---

# Convert protocol numbers to string labels to match UNSW
df_pcap["protocol_m"] = df_pcap["protocol"].astype(str)

# Convert stime to integer seconds (UNSW flow timestamps are in integer seconds)
df_pcap["stime_sec"] = df_pcap["stime"].astype(int)

# Replace NaNs in ports with -1 (consistent dummy value)
df_pcap["sport"] = df_pcap["sport"].fillna(-1).astype(int)
df_pcap["dsport"] = df_pcap["dsport"].fillna(-1).astype(int)

df_pcap.head()


,srcip,dstip,ttl,protocol,sport,dsport,stime,frame_num,payload,protocol_m,stime_sec
0,10.40.85.1,224.0.0.5,1.0,89,-1,-1,1.421927e+09,0,0201002cc0a8f1f300000000ead8000000000000000000...,89,1421927376
1,10.40.182.1,224.0.0.5,1.0,89,-1,-1,1.421927e+09,1,0201002cc0a8f1f30000000089d8000000000000000000...,89,1421927376
2,192.168.241.243,192.168.241.243,64.0,1,-1,-1,1.421927e+09,2,030154460000000045000148000040004011d52dc0a8f1...,1,1421927381
3,10.40.85.1,224.0.0.5,1.0,89,-1,-1,1.421927e+09,3,0201002cc0a8f1f300000000ead8000000000000000000...,89,1421927386
4,10.40.182.1,224.0.0.5,1.0,89,-1,-1,1.421927e+09,4,0201002cc0a8f1f30000000089d8000000000000000000...,89,1421927386


In [38]:
# Look at the key columns in the flow table
print("Flow columns (first 25):", df_flow.columns[:25].tolist())

# Show a few rows of the important-looking columns
cols_to_peek = [c for c in ["srcip", "sport", "dstip", "dsport", "proto", "stime", "ltime", "dur", "attack_cat", "label"] if c in df_flow.columns]
print("\nPreview columns:", cols_to_peek)

df_flow[cols_to_peek].head()


Flow columns (first 25): ['59.166.0.0', '1390', '149.171.126.6', '53', 'udp', 'CON', '0.001055', '132', '164', '31', '29', '0', '0.1', 'dns', '500473.9375', '621800.9375', '2', '2.1', '0.2', '0.3', '0.4', '0.5', '66', '82', '0.6']

Preview columns: []


""
0
1
2
3
4


In [42]:
import pandas as pd

feat = pd.read_csv("/content/NUSW-NB15_features.csv", encoding="latin1")

print("Feature file shape:", feat.shape)
feat.head()


Feature file shape: (49, 4)


,No.,Name,Type,Description
0,1,srcip,nominal,Source IP address
1,2,sport,integer,Source port number
2,3,dstip,nominal,Destination IP address
3,4,dsport,integer,Destination port number
4,5,proto,nominal,Transaction protocol


In [43]:
# Extract feature names
feature_names = feat["Name"].tolist()
print("Number of feature names:", len(feature_names))
print("First 10:", feature_names[:10])

# Read the 4 UNSW CSV files with the correct header
flow1 = pd.read_csv("/content/UNSW-NB15_1.csv", names=feature_names, header=None)
flow2 = pd.read_csv("/content/UNSW-NB15_2.csv", names=feature_names, header=None)
flow3 = pd.read_csv("/content/UNSW-NB15_3.csv", names=feature_names, header=None)
flow4 = pd.read_csv("/content/UNSW-NB15_4.csv", names=feature_names, header=None)

# Combine into one flow table
df_flow = pd.concat([flow1, flow2, flow3, flow4], ignore_index=True)

print("Total flows:", len(df_flow))
df_flow.head()


Number of feature names: 49
First 10: ['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur', 'sbytes', 'dbytes', 'sttl']


/tmp/ipython-input-1819692449.py:7: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  flow1 = pd.read_csv("/content/UNSW-NB15_1.csv", names=feature_names, header=None)
/tmp/ipython-input-1819692449.py:8: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  flow2 = pd.read_csv("/content/UNSW-NB15_2.csv", names=feature_names, header=None)


Total flows: 2540047


,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,...,0,3,7,1,3,1,1,1,NaN,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0,2,4,2,3,1,1,2,NaN,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0,12,8,1,2,2,1,1,NaN,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0,6,9,1,1,1,1,1,NaN,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,0,7,9,1,1,1,1,1,NaN,0


In [46]:
import numpy as np

# 1) Define the flow key
flow_key = ["srcip", "dstip", "sport", "dsport", "protocol"]

# Check all columns exist
print("Missing columns:", [c for c in flow_key if c not in df_pcap.columns])

# 2) Create a numeric flow_id based on the 5-tuple
df_pcap["flow_id"] = df_pcap.groupby(flow_key).ngroup()

print("Number of packets:", len(df_pcap))
print("Number of unique flows:", df_pcap["flow_id"].nunique())

# See first few rows with the flow_id attached
df_pcap[flow_key + ["frame_num", "stime", "flow_id"]].head(10)


Missing columns: []
Number of packets: 20000
Number of unique flows: 2283


,srcip,dstip,sport,dsport,protocol,frame_num,stime,flow_id
0,10.40.85.1,224.0.0.5,-1,-1,89,0,1.421927e+09,6
1,10.40.182.1,224.0.0.5,-1,-1,89,1,1.421927e+09,2
2,192.168.241.243,192.168.241.243,-1,-1,1,2,1.421927e+09,1175
3,10.40.85.1,224.0.0.5,-1,-1,89,3,1.421927e+09,6
4,10.40.182.1,224.0.0.5,-1,-1,89,4,1.421927e+09,2
5,192.168.241.243,192.168.241.243,-1,-1,1,5,1.421927e+09,1175
6,10.40.85.1,224.0.0.5,-1,-1,89,6,1.421927e+09,6
7,10.40.182.1,224.0.0.5,-1,-1,89,7,1.421927e+09,2
8,192.168.241.243,192.168.241.243,-1,-1,1,8,1.421927e+09,1175
9,10.40.85.1,224.0.0.5,-1,-1,89,9,1.421927e+09,6


In [47]:
import numpy as np

flow_key = ["srcip", "dstip", "sport", "dsport", "protocol"]

# For each flow_id, count how many distinct values there are for each key column
n_unique_per_flow = df_pcap.groupby("flow_id")[flow_key].nunique()

# A "bad" flow would have more than 1 unique value in any of these columns
bad_flows = n_unique_per_flow[(n_unique_per_flow > 1).any(axis=1)]

print("Total flows:", n_unique_per_flow.shape[0])
print("Flows with inconsistent 5-tuple:", bad_flows.shape[0])

# Show a few if there are any
if not bad_flows.empty:
    print("\nExample inconsistent flows:\n", bad_flows.head())


Total flows: 2283
Flows with inconsistent 5-tuple: 0


In [51]:
def check_flow_order(group):
    stimes = group["stime"].values
    frames = group["frame_num"].values

    time_ok = np.all(stimes == np.sort(stimes))
    frame_ok = np.all(frames == np.sort(frames))

    return time_ok and frame_ok

order_ok = df_pcap.groupby("flow_id").apply(check_flow_order)

num_bad = (~order_ok).sum()

print("Flows with incorrect ordering:", num_bad)


Flows with incorrect ordering: 0


/tmp/ipython-input-1689116525.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  order_ok = df_pcap.groupby("flow_id").apply(check_flow_order)


In [49]:
# # Make a shuffled copy of the packet dataframe
# df_shuffled = df_pcap.sample(frac=1.0, random_state=42).reset_index(drop=True)

# def check_flow_order(group):
#     stimes = group["stime"].values
#     frames = group["frame_num"].values

#     time_ok = np.all(stimes == np.sort(stimes))
#     frame_ok = np.all(frames == np.sort(frames))
#     return time_ok and frame_ok

# order_ok_shuffled = df_shuffled.groupby("flow_id").apply(check_flow_order)
# num_bad_shuffled = (~order_ok_shuffled).sum()

# print("Flows with incorrect ordering AFTER SHUFFLING:", num_bad_shuffled)


Flows with incorrect ordering AFTER SHUFFLING: 1965


/tmp/ipython-input-911252713.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  order_ok_shuffled = df_shuffled.groupby("flow_id").apply(check_flow_order)


In [50]:
# import numpy as np

# # 1) Copy the original packet dataframe
# df_corrupt = df_pcap.copy()

# # 2) Pick an example flow_id
# bad_flow_id = df_corrupt["flow_id"].iloc[0]
# print("Corrupting flow_id:", bad_flow_id)

# # 3) Change the sport of ONE packet in that flow
# idx_to_corrupt = df_corrupt[df_corrupt["flow_id"] == bad_flow_id].index[0]
# original_sport = df_corrupt.at[idx_to_corrupt, "sport"]
# df_corrupt.at[idx_to_corrupt, "sport"] = original_sport + 1  # make it different

# print("Original sport:", original_sport, " -> New sport:", df_corrupt.at[idx_to_corrupt, "sport"])

# # 4) Re-run the 5-tuple consistency check
# flow_key = ["srcip", "dstip", "sport", "dsport", "protocol"]
# n_unique_per_flow_corrupt = df_corrupt.groupby("flow_id")[flow_key].nunique()
# bad_flows_corrupt = n_unique_per_flow_corrupt[(n_unique_per_flow_corrupt > 1).any(axis=1)]

# print("Total flows:", n_unique_per_flow_corrupt.shape[0])
# print("Flows with inconsistent 5-tuple AFTER CORRUPTION:", bad_flows_corrupt.shape[0])
# print("Does our chosen flow_id appear in bad flows?", bad_flow_id in bad_flows_corrupt.index)


Corrupting flow_id: 6
Original sport: -1  -> New sport: 0
Total flows: 2283
Flows with inconsistent 5-tuple AFTER CORRUPTION: 1
Does our chosen flow_id appear in bad flows? True


In [52]:
df_pcap.head()
df_pcap.groupby("flow_id")[["srcip","dstip","sport","dsport","protocol"]].nunique().max()


,0
srcip,1
dstip,1
sport,1
dsport,1
protocol,1


In [53]:
# Pick any example flow_id (here we just take the first one)
example_flow_id = df_pcap["flow_id"].iloc[0]
print("Example flow_id:", example_flow_id)

flow_example = df_pcap[df_pcap["flow_id"] == example_flow_id] \
    .sort_values(["stime", "frame_num"])

flow_example[["flow_id", "frame_num", "stime", "srcip", "sport", "dstip", "dsport", "protocol"]].head(30)


Example flow_id: 6


,flow_id,frame_num,stime,srcip,sport,dstip,dsport,protocol
0,6,0,1.421927e+09,10.40.85.1,-1,224.0.0.5,-1,89
3,6,3,1.421927e+09,10.40.85.1,-1,224.0.0.5,-1,89
6,6,6,1.421927e+09,10.40.85.1,-1,224.0.0.5,-1,89
9,6,9,1.421927e+09,10.40.85.1,-1,224.0.0.5,-1,89
399,6,399,1.421927e+09,10.40.85.1,-1,224.0.0.5,-1,89
1562,6,1562,1.421927e+09,10.40.85.1,-1,224.0.0.5,-1,89
2948,6,2948,1.421927e+09,10.40.85.1,-1,224.0.0.5,-1,89
